In [1]:
# import spacy
import dateparser
from dateutil.parser import parse

In [2]:
# my notebook is crashing if I don't include this cell
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# !conda install -c conda-forge spacy
# !python -m spacy download en_core_web_sm

In [4]:
import spacy

/Users/filipilievski/opt/anaconda3/envs/wikiEnv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
nlp = spacy.load("en_core_web_sm")

In [6]:
def extract_dates(blob):
    doc=nlp(blob)
    dates=[]
    dates_norm=[]
    for ent in doc.ents:
        if ent.label_=='DATE':
            dates.append(ent.text)
            dates_norm.append(dateparser.parse(str(ent.text)))
    return dates, dates_norm

In [7]:
blob="On August 31st, 2010, the face was included in an updated rage template image post in the f7u12 subreddit.[15] Redditor wallish[16] replied to the thread referring to the face as \"Asian Laughing Guy.\""
d=extract_dates(blob)

/Users/filipilievski/opt/anaconda3/envs/wikiEnv/lib/python3.8/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


In [8]:
for date in d:
    print(dateparser.parse(str(date)))

2010-08-31 00:00:00
None


## Using spacy to extract dates from KYM

In [9]:
import json
with open('kym.json', 'r') as f:
    k=json.load(f)

In [10]:
len(k)

28799

In [11]:
example=k[1]['content']['origin']['text'][0]

In [12]:
dates={}
from tqdm import tqdm
for i in tqdm(range(len(k))):
    if 'content' in k[i] and 'origin' in k[i]['content'] and 'text' in k[i]['content']['origin']:
        example=k[i]['content']['origin']['text'][0]
        d, dnorm=extract_dates(example)
        dates[str(i)]=d
#        print(d)

100%|██████████| 28799/28799 [02:30<00:00, 190.74it/s]


In [13]:
len(dates)

9223

In [14]:
from collections import Counter
sizes=[]
for dt in dates:
    sizes.append(len(dt))

In [15]:
Counter(sizes)

Counter({1: 10, 2: 39, 3: 257, 4: 3090, 5: 5827})

In [19]:
with open('kym_dates.json', 'w') as w:
    json.dump(dates, w)